<a href="https://colab.research.google.com/github/norman-AI-2025/hackathon-2025/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit pandas transformers torch pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.8 MB/s eta 0:00:00


In [2]:
loan_risk_app_code = """
import pandas as pd
import streamlit as st
from typing import Callable
import numpy as np
import time

# --- CONSTANTS AND CONFIGURATION ---

# 1. Page and State Management
PAGES = {
    1: "Applicant Info",
    2: "Guarantor Info",
    3: "Loan Details",
    4: "Results & Text Essay"
}

# 2. Categorical Options
GENDER_OPTIONS = ["Male", "Female", "Other"]
MARITAL_OPTIONS = ["Single", "Married", "Divorced", "Widowed"]
EDUCATION_OPTIONS = ["Graduate", "Non-Graduate"]
EMPLOYMENT_OPTIONS = ["Salaried", "Self-Employed", "Business Owner", "Unemployed"]
RELATIONSHIP_OPTIONS = ["Spouse", "Parent", "Sibling", "Friend", "Company"]
PURPOSE_OPTIONS = ["Car Loan", "Education", "Personal", "Renovation", "Business Expansion", "Other"]
COLLATERAL_OPTIONS = ["Real Estate", "Vehicle", "Fixed Deposit", "None"]


# --- 1. Text Analysis Setup (Unchanged from previous versions) ---

@st.cache_resource
def load_sentiment_model():
    \"\"\"Initializes the sentiment analysis model.\"\"\"
    try:
        from transformers import pipeline
        classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
        return classifier
    except Exception as e:
        st.warning(
            f"⚠️ **Warning: Could not initialize sentiment model.** Text analysis will be skipped. "
            f"Install `transformers` and `torch` to enable full text analysis. Error: {e}"
        )
        return None

classifier = load_sentiment_model()

def compute_text_score(text: str) -> float:
    \"\"\"Computes a risk score (0=Low Risk, 100=High Risk) based on text sentiment.\"\"\"
    if classifier is None or pd.isna(text) or str(text).strip() == "":
        return 50.0

    try:
        result = classifier(str(text))[0]
        score = result['score'] * 100
        return score if result['label'] == 'NEGATIVE' else 100 - score
    except Exception:
        return 50.0

# --- 2. Numeric Model (UPDATED FOR NEW FEATURES) ---

def compute_numeric_risk_scores(data: dict) -> dict:
    \"\"\"
    Computes a risk score (0 to 100) based on all Applicant, Guarantor, and Loan features.

    Weights (Total = 100):
    - DTI & Income: 35
    - Credit/Commitments: 35
    - Loan & Term: 20
    - Demographics: 10
    \"\"\"

    # --- 1. Feature Engineering & Pre-calculations ---

    # Combined Income and DTI
    total_applicant_income = data['app_monthly_income'] + data['app_other_monthly_income']
    dti_ratio = (data['app_total_monthly_loan_repayment'] + data['app_other_fixed_monthly_commitments']) / (total_applicant_income + 1e-6) # Avoid division by zero

    # Credit Health
    credit_limit = data['app_total_credit_limit'] if data['app_total_credit_limit'] > 0 else 1e-6
    credit_utilization = data['app_credit_card_outstanding'] / credit_limit

    # Loan-to-Value (LTV)
    collateral_value = data['loan_collateral_value'] if data['loan_collateral_value'] > 0 else 1e-6
    ltv_ratio = data['loan_amount_requested'] / collateral_value if data['loan_is_secured'] else 0.0

    # Initialize score and risk breakdown
    numeric_score = 0
    risk_breakdown = {}

    # --- 2. Risk Indicator Calculation (0=Low Risk, 100=High Risk) ---

    # A. INCOME & DTI RISK (Weight: 35)

    # DTI: High DTI is high risk. (Max practical DTI is usually 50%)
    dti_risk = min(dti_ratio / 0.5, 1.0) * 100
    numeric_score += dti_risk * 0.20 # 20% weight on DTI
    risk_breakdown['DTI_Risk'] = dti_risk * 0.20

    # Combined Income: Low income is high risk (assuming max income of 150k for full risk reduction)
    income_risk = max(0, 1 - (total_applicant_income / 150000)) * 100
    numeric_score += income_risk * 0.15 # 15% weight on Income
    risk_breakdown['Income_Risk'] = income_risk * 0.15

    # B. CREDIT & COMMITMENTS RISK (Weight: 35)

    # Credit History Proxy: We use the presence of car/personal/mortgage loan commitments as weak indicators.
    commitments_risk = sum([
        10 if data['app_has_mortgage'] else 0,
        15 if data['app_has_car_loan'] else 0,
        15 if data['app_has_personal_loan'] else 0,
    ]) / 40 * 100 # Normalize to 100
    numeric_score += commitments_risk * 0.15 # 15% weight on loan commitments
    risk_breakdown['Commitments_Risk'] = commitments_risk * 0.15

    # Credit Utilization: High utilization (>50%) is high risk
    utilization_risk = min(credit_utilization / 0.5, 1.0) * 100
    numeric_score += utilization_risk * 0.20 # 20% weight on Utilization
    risk_breakdown['Credit_Utilization_Risk'] = utilization_risk * 0.20

    # C. LOAN & TERM RISK (Weight: 20)

    # Loan Term: Longer term (>360 months) is higher risk
    term_risk = min(data['loan_tenure_months'] / 360, 1.0) * 100
    numeric_score += term_risk * 0.10 # 10% weight on Term
    risk_breakdown['Loan_Term_Risk'] = term_risk * 0.10

    # Secured/LTV Risk: Unsecured/high LTV (>90%) is high risk
    ltv_risk = 0.0
    if data['loan_is_secured']:
        ltv_risk = min(ltv_ratio / 0.9, 1.0) * 100 # Risk scales with LTV
    else:
        ltv_risk = 70 # Arbitrary high risk for unsecured

    numeric_score += ltv_risk * 0.10 # 10% weight on LTV/Secured status
    risk_breakdown['LTV_Secured_Risk'] = ltv_risk * 0.10

    # D. DEMOGRAPHICS & EMPLOYMENT RISK (Weight: 10)

    # Employment Stability: Low years in job is high risk
    stability_risk = max(0, 1 - (data['app_years_in_job'] / 10)) * 100 # 10+ years is low risk
    numeric_score += stability_risk * 0.05 # 5% weight on Stability
    risk_breakdown['Stability_Risk'] = stability_risk * 0.05

    # Age/Dependents: High dependents/low age is high risk (simplified)
    demographic_risk = (data['app_num_dependents'] / 5 + max(0, 50 - data['app_age']) / 50) * 50 # Max 5 dependents, minimum age risk at 50
    demographic_risk = min(demographic_risk, 100)
    numeric_score += demographic_risk * 0.05 # 5% weight on Demographics
    risk_breakdown['Demographic_Risk'] = demographic_risk * 0.05

    # Guarantor adjustment (Bonus reduction if a strong guarantor exists)
    guarantor_bonus = 0
    if data['guar_exists'] and data['guar_monthly_income'] > data['app_monthly_income'] * 0.5:
        guarantor_bonus = 10 # Reduce score by 10 points if guarantor is strong

    # Final numeric score (0-100)
    final_numeric_score = np.clip(numeric_score - guarantor_bonus, 0, 100)

    data['numeric_score'] = final_numeric_score
    data['risk_breakdown'] = risk_breakdown
    return data

# --- 3. Fusion Model (Unchanged from previous versions) ---

def compute_fusion_risk(data: dict, alpha: float = 0.5) -> dict:
    \"\"\"Fuses numeric and text risk scores into a single final risk score (0-100).\"\"\"

    # 1) Compute text_score
    text_score = compute_text_score(data['loan_essay_text'])
    data['text_score'] = text_score

    # 2) Fusion
    alpha = np.clip(alpha, 0.0, 1.0)
    numeric_score = data['numeric_score']

    # Weighted Average for Final Risk Score
    data["risk_score"] = (alpha * numeric_score) + ((1 - alpha) * text_score)

    # 3) Categorize the final risk score
    score = data["risk_score"]
    if score >= 70:
        data["risk_category"] = "High Risk"
    elif score >= 30:
        data["risk_category"] = "Medium Risk"
    else:
        data["risk_category"] = "Low Risk"

    return data


# --- 4. Streamlit UI Components (Multi-Page Flow) ---

def init_session_state():
    \"\"\"Initializes necessary session state variables.\"\"\"
    if 'current_page' not in st.session_state:
        st.session_state.current_page = 1
    if 'form_data' not in st.session_state:
        # Initialize all expected columns for a complete data row
        st.session_state.form_data = {
            # Page 1: Applicant Info
            'app_age': 30, 'app_gender': 'Male', 'app_marital_status': 'Single', 'app_num_dependents': 0,
            'app_education': 'Graduate', 'app_employment_status': 'Salaried', 'app_years_in_job': 5,
            'app_total_work_experience': 10, 'app_monthly_income': 50000, 'app_other_monthly_income': 0,
            'app_has_mortgage': False, 'app_has_car_loan': False, 'app_has_personal_loan': False,
            'app_has_credit_card': False, 'app_num_credit_cards': 1, 'app_total_credit_limit': 100000,
            'app_credit_card_outstanding': 10000, 'app_total_monthly_loan_repayment': 5000,
            'app_other_fixed_monthly_commitments': 0,

            # Page 2: Guarantor Info
            'guar_exists': False, 'guar_relationship': 'Spouse', 'guar_age': 30,
            'guar_employment_status': 'Salaried', 'guar_monthly_income': 0, 'guar_other_monthly_income': 0,
            'guar_total_monthly_loan_repayment': 0, 'guar_has_credit_card': False,
            'guar_num_credit_cards': 0, 'guar_total_credit_limit': 0, 'guar_credit_card_outstanding': 0,

            # Page 3: Loan Details
            'loan_amount_requested': 150000, 'loan_tenure_months': 360, 'loan_purpose': 'Personal',
            'loan_is_secured': False, 'loan_collateral_type': 'None', 'loan_collateral_value': 0,
            'loan_essay_text': "I am seeking this loan to consolidate existing high-interest debt and invest in my education, which will significantly increase my future earning potential. I have a stable job and a clear, detailed plan for repayment.",

            # Results
            'numeric_score': 0.0, 'text_score': 0.0, 'risk_score': 0.0, 'risk_category': 'N/A',
            'risk_breakdown': {}
        }


def page_applicant_info():
    \"\"\"Form fields for Applicant Demographic and Commitment info (Page 1).\"\"\"
    st.header(f"Page 1: Applicant Info ({PAGES[1]})")

    st.subheader("Demographics & Employment")
    col1, col2, col3 = st.columns(3)
    st.session_state.form_data['app_age'] = col1.number_input("Age (Years)", min_value=18, max_value=100, value=st.session_state.form_data['app_age'], key="app_age")
    st.session_state.form_data['app_gender'] = col2.selectbox("Gender", options=GENDER_OPTIONS, index=GENDER_OPTIONS.index(st.session_state.form_data['app_gender']), key="app_gender")
    st.session_state.form_data['app_marital_status'] = col3.selectbox("Marital Status", options=MARITAL_OPTIONS, index=MARITAL_OPTIONS.index(st.session_state.form_data['app_marital_status']), key="app_marital_status")

    col4, col5, col6 = st.columns(3)
    st.session_state.form_data['app_num_dependents'] = col4.number_input("Number of Dependents", min_value=0, max_value=10, value=st.session_state.form_data['app_num_dependents'], key="app_num_dependents")
    st.session_state.form_data['app_education'] = col5.selectbox("Education Level", options=EDUCATION_OPTIONS, index=EDUCATION_OPTIONS.index(st.session_state.form_data['app_education']), key="app_education")
    st.session_state.form_data['app_employment_status'] = col6.selectbox("Employment Status", options=EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['app_employment_status']), key="app_employment_status")

    col7, col8 = st.columns(2)
    st.session_state.form_data['app_years_in_job'] = col7.number_input("Years in Current Job", min_value=0, max_value=50, value=st.session_state.form_data['app_years_in_job'], key="app_years_in_job")
    st.session_state.form_data['app_total_work_experience'] = col8.number_input("Total Work Experience (Years)", min_value=0, max_value=50, value=st.session_state.form_data['app_total_work_experience'], key="app_total_work_experience")

    st.subheader("Income & Financial Commitments")
    col9, col10 = st.columns(2)
    st.session_state.form_data['app_monthly_income'] = col9.number_input("Monthly Income ($)", min_value=0, value=st.session_state.form_data['app_monthly_income'], step=1000, key="app_monthly_income")
    st.session_state.form_data['app_other_monthly_income'] = col10.number_input("Other Monthly Income ($)", min_value=0, value=st.session_state.form_data['app_other_monthly_income'], step=500, key="app_other_monthly_income")

    st.markdown("---")
    st.subheader("Existing Commitments")

    col_loan1, col_loan2, col_loan3 = st.columns(3)
    st.session_state.form_data['app_has_mortgage'] = col_loan1.checkbox("Has Mortgage?", value=st.session_state.form_data['app_has_mortgage'], key="app_has_mortgage")
    st.session_state.form_data['app_has_car_loan'] = col_loan2.checkbox("Has Car Loan?", value=st.session_state.form_data['app_has_car_loan'], key="app_has_car_loan")
    st.session_state.form_data['app_has_personal_loan'] = col_loan3.checkbox("Has Personal Loan?", value=st.session_state.form_data['app_has_personal_loan'], key="app_has_personal_loan")

    st.session_state.form_data['app_has_credit_card'] = st.checkbox("Has Credit Card?", value=st.session_state.form_data['app_has_credit_card'], key="app_has_credit_card")

    col_credit1, col_credit2, col_credit3 = st.columns(3)
    st.session_state.form_data['app_num_credit_cards'] = col_credit1.number_input("Number of Credit Cards", min_value=0, value=st.session_state.form_data['app_num_credit_cards'], key="app_num_credit_cards")
    st.session_state.form_data['app_total_credit_limit'] = col_credit2.number_input("Total Credit Limit ($)", min_value=0, value=st.session_state.form_data['app_total_credit_limit'], step=10000, key="app_total_credit_limit")
    st.session_state.form_data['app_credit_card_outstanding'] = col_credit3.number_input("Credit Card Outstanding ($)", min_value=0, value=st.session_state.form_data['app_credit_card_outstanding'], step=1000, key="app_credit_card_outstanding")

    col_repay1, col_repay2 = st.columns(2)
    st.session_state.form_data['app_total_monthly_loan_repayment'] = col_repay1.number_input("Total Monthly Loan Repayment ($)", min_value=0, value=st.session_state.form_data['app_total_monthly_loan_repayment'], step=500, key="app_total_monthly_loan_repayment")
    st.session_state.form_data['app_other_fixed_monthly_commitments'] = col_repay2.number_input("Other Fixed Monthly Commitments ($)", min_value=0, value=st.session_state.form_data['app_other_fixed_monthly_commitments'], step=500, help="E.g., child support, insurance, etc.", key="app_other_fixed_monthly_commitments")


def page_guarantor_info():
    \"\"\"Form fields for Guarantor info (Page 2).\"\"\"
    st.header(f"Page 2: Guarantor Info ({PAGES[2]})")

    st.session_state.form_data['guar_exists'] = st.radio(
        "Do you have a guarantor?",
        options=[True, False],
        format_func=lambda x: "Yes" if x else "No",
        index=0 if st.session_state.form_data['guar_exists'] else 1,
        key="guar_exists"
    )

    if st.session_state.form_data['guar_exists']:
        st.subheader("Guarantor Details")
        col1, col2, col3 = st.columns(3)
        st.session_state.form_data['guar_relationship'] = col1.selectbox("Relationship to Applicant", options=RELATIONSHIP_OPTIONS, index=RELATIONSHIP_OPTIONS.index(st.session_state.form_data['guar_relationship']), key="guar_relationship")
        st.session_state.form_data['guar_age'] = col2.number_input("Age (Years)", min_value=18, max_value=100, value=st.session_state.form_data['guar_age'], key="guar_age")
        st.session_state.form_data['guar_employment_status'] = col3.selectbox("Employment Status", options=EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['guar_employment_status']), key="guar_employment_status")

        col4, col5 = st.columns(2)
        st.session_state.form_data['guar_monthly_income'] = col4.number_input("Monthly Income ($)", min_value=0, value=st.session_state.form_data['guar_monthly_income'], step=1000, key="guar_monthly_income")
        st.session_state.form_data['guar_other_monthly_income'] = col5.number_input("Other Monthly Income ($)", min_value=0, value=st.session_state.form_data['guar_other_monthly_income'], step=500, key="guar_other_monthly_income")

        st.session_state.form_data['guar_total_monthly_loan_repayment'] = st.number_input("Total Monthly Loan Repayment ($)", min_value=0, value=st.session_state.form_data['guar_total_monthly_loan_repayment'], step=500, key="guar_total_monthly_loan_repayment")

        st.session_state.form_data['guar_has_credit_card'] = st.checkbox("Has Credit Card?", value=st.session_state.form_data['guar_has_credit_card'], key="guar_has_credit_card_check")

        col_credit1, col_credit2, col_credit3 = st.columns(3)
        st.session_state.form_data['guar_num_credit_cards'] = col_credit1.number_input("Number of Credit Cards", min_value=0, value=st.session_state.form_data['guar_num_credit_cards'], key="guar_num_credit_cards")
        st.session_state.form_data['guar_total_credit_limit'] = col_credit2.number_input("Total Credit Limit ($)", min_value=0, value=st.session_state.form_data['guar_total_credit_limit'], step=10000, key="guar_total_credit_limit")
        st.session_state.form_data['guar_credit_card_outstanding'] = col_credit3.number_input("Credit Card Outstanding ($)", min_value=0, value=st.session_state.form_data['guar_credit_card_outstanding'], step=1000, key="guar_credit_card_outstanding")

    else:
        # If no guarantor, set all guarantor related risk inputs to 0 for computation
        for key in list(st.session_state.form_data.keys()):
            if key.startswith('guar_') and key != 'guar_exists':
                # For numbers, set to 0. For strings/selections, set to a safe default.
                if isinstance(st.session_state.form_data[key], (int, float)):
                    st.session_state.form_data[key] = 0
                elif isinstance(st.session_state.form_data[key], bool):
                    st.session_state.form_data[key] = False
                elif key == 'guar_relationship':
                    st.session_state.form_data[key] = 'None'


def page_loan_details():
    \"\"\"Form fields for Loan details and essay text (Page 3).\"\"\"
    st.header(f"Page 3: Loan Details ({PAGES[3]})")

    col1, col2 = st.columns(2)
    st.session_state.form_data['loan_amount_requested'] = col1.number_input("Loan Amount Requested ($)", min_value=1000, value=st.session_state.form_data['loan_amount_requested'], step=10000, key="loan_amount_requested")
    st.session_state.form_data['loan_tenure_months'] = col2.number_input("Loan Tenure (Months)", min_value=12, max_value=480, value=st.session_state.form_data['loan_tenure_months'], step=12, key="loan_tenure_months")

    st.session_state.form_data['loan_purpose'] = st.selectbox("Purpose", options=PURPOSE_OPTIONS, index=PURPOSE_OPTIONS.index(st.session_state.form_data['loan_purpose']), key="loan_purpose")

    st.session_state.form_data['loan_is_secured'] = st.radio(
        "Is this a Secured Loan?",
        options=[True, False],
        format_func=lambda x: "Yes (with Collateral)" if x else "No (Unsecured)",
        index=0 if st.session_state.form_data['loan_is_secured'] else 1,
        key="loan_is_secured"
    )

    if st.session_state.form_data['loan_is_secured']:
        st.subheader("Collateral Details")
        col_coll1, col_coll2 = st.columns(2)
        collateral_options_secured = [c for c in COLLATERAL_OPTIONS if c != 'None']
        current_collateral = st.session_state.form_data['loan_collateral_type']
        if current_collateral not in collateral_options_secured:
            current_collateral = collateral_options_secured[0]

        st.session_state.form_data['loan_collateral_type'] = col_coll1.selectbox("Collateral Type", options=collateral_options_secured, index=collateral_options_secured.index(current_collateral), key="loan_collateral_type")
        st.session_state.form_data['loan_collateral_value'] = col_coll2.number_input("Collateral Value ($)", min_value=1000, value=st.session_state.form_data['loan_collateral_value'] if st.session_state.form_data['loan_collateral_value'] > 0 else st.session_state.form_data['loan_amount_requested'], step=10000, key="loan_collateral_value")
    else:
        st.session_state.form_data['loan_collateral_type'] = 'None'
        st.session_state.form_data['loan_collateral_value'] = 0

    st.subheader("Loan Purpose Essay (Text Analysis Input)")
    st.session_state.form_data['loan_essay_text'] = st.text_area(
        "Please type your essay explaining why you want the loan:",
        value=st.session_state.form_data['loan_essay_text'],
        height=200,
        help="The sentiment of this text will contribute to the final risk score."
    )


def page_results(alpha_weight):
    \"\"\"Triggers risk calculation and displays final results (Page 4).\"\"\"
    st.header(f"Page 4: Risk Assessment ({PAGES[4]})")

    if st.button("Submit & Get Final Risk Score", type="primary"):
        with st.spinner("Calculating numeric risk, analyzing essay sentiment, and fusing results..."):

            # 1. Run Numeric Model (Feature Engineering and Scoring)
            processed_data = compute_numeric_risk_scores(st.session_state.form_data)

            # 2. Run Fusion Model (Includes Text Score calculation)
            final_results = compute_fusion_risk(processed_data, alpha=alpha_weight)
            st.session_state.form_data.update(final_results) # Update session state with results

            # Delay slightly for dramatic effect
            time.sleep(1.0)

        st.success("Analysis Complete!")

        result = st.session_state.form_data

        st.markdown("#### Final Assessment")
        col_res1, col_res2, col_res3, col_res4 = st.columns(4)

        col_res1.metric("Risk Category", result['risk_category'])
        col_res2.metric("Final Risk Score (0-100)", f"{result['risk_score']:.2f}")
        col_res3.metric("Numeric Score (Weighted)", f"{result['numeric_score']:.2f}")
        col_res4.metric("Text Sentiment Score", f"{result['text_score']:.2f}")

        st.markdown(\"\"\"
        <style>
        div[data-testid="stMetricValue"] {
            font-size: 32px;
        }
        </style>
        \"\"\", unsafe_allow_html=True)

        st.markdown("---")
        st.subheader("Detailed Numeric Risk Breakdown (Total = 100)")

        # Displaying the risk breakdown created in compute_numeric_risk_scores
        breakdown_df = pd.DataFrame(
            {'Risk Component': list(result['risk_breakdown'].keys()),
             'Contribution to Score': [f"{v:.2f}" for v in result['risk_breakdown'].values()]}
        )

        st.dataframe(breakdown_df, hide_index=True, use_container_width=True)
        st.info(f"The final numeric score is {result['numeric_score']:.2f}, factoring in a **Guarantor Bonus** of up to 10 points if the guarantor is strong.")


# --- MAIN APP EXECUTION ---

def main_app():
    \"\"\"Controls the overall app flow and navigation.\"\"\"
    init_session_state()

    st.set_page_config(page_title="Loan Risk Application Form", layout="wide")
    st.title("🏦 Comprehensive Loan Risk Application")
    st.markdown("Please fill out the following pages to receive a combined Numeric + Text risk score.")

    # --- Sidebar Configuration ---
    st.sidebar.header("Navigation & Configuration")

    # Page navigation buttons
    col_nav1, col_nav2 = st.sidebar.columns(2)

    if st.session_state.current_page > 1:
        if col_nav1.button("← Previous"):
            st.session_state.current_page -= 1
            st.rerun()

    if st.session_state.current_page < len(PAGES):
        if col_nav2.button("Next →"):
            st.session_state.current_page += 1
            st.rerun()

    # Weight Slider (Active on all pages but primarily controls the final page)
    alpha_weight = st.sidebar.slider(
        "Numeric Score Weight (α)",
        min_value=0.0,
        max_value=1.0,
        value=0.5,
        step=0.05,
        help="Controls the balance: Numeric (α) vs. Text (1 - α)."
    )
    st.sidebar.markdown(f"**Numeric Weight:** `{alpha_weight:.2f}` | **Text Weight:** `{1 - alpha_weight:.2f}`")

    # --- Page Renderer ---

    page_func = {
        1: page_applicant_info,
        2: page_guarantor_info,
        3: page_loan_details,
        4: lambda: page_results(alpha_weight)
    }.get(st.session_state.current_page)

    page_func()

    st.markdown("---")
    st.write(f"Current Step: **{st.session_state.current_page}/{len(PAGES)} - {PAGES[st.session_state.current_page]}**")

    # --- CSV Upload for Batch Processing (Still needed for the original functionality) ---
    if st.session_state.current_page == len(PAGES):
        st.subheader("2. Batch CSV Upload (Original Functionality)")
        st.info("The single-page form above is for manual entry. Use the uploader below for bulk analysis.")

        main_csv_file = st.file_uploader(
            "Upload Main Applicant Data CSV (Required Columns: app_age, app_gender, app_monthly_income, etc.)",
            type="csv",
            help="Upload a CSV with all the columns specified in the form (app_..., guar_..., loan_...)."
        )

        guarantor_csv_file = st.file_uploader(
            "Upload Guarantor Data CSV (Optional - For Review Only)",
            type="csv",
            help="This data is loaded for manual review. The risk model uses data merged into the main CSV row."
        )

        # NOTE: Batch processing for the new schema is pending full training data schema confirmation.
        if main_csv_file is not None:
             st.warning("Batch processing functionality will be fully implemented once the final training data schema is confirmed.")


main_app()
"""
with open('streamlit_loan_risk_app.py', 'w') as f:
    f.write(loan_risk_app_code)

print("✅ Application file 'streamlit_loan_risk_app.py' saved.")


✅ Application file 'streamlit_loan_risk_app.py' saved.


In [5]:
from pyngrok import ngrok
import subprocess

try:
    # Terminate any existing ngrok tunnels
    ngrok.kill()

    # Start ngrok tunnel for Streamlit's default port (8501)
    port = 8501
    public_url = ngrok.connect(port).public_url

    print(f"\n🌍 Your Streamlit app is running at: {public_url}")
    print("Click the link above to open the application in a new tab.")

    # 4. RUN STREAMLIT
    # Run Streamlit command in the background
    command = f"streamlit run streamlit_loan_risk_app.py --server.port {port} --server.enableCORS false --server.enableXsrfProtection false > /dev/null"
    subprocess.Popen(command, shell=True)

except Exception as e:
    print(f"\n❌ An error occurred during ngrok setup or Streamlit execution: {e}")
    print("If you encounter 'tunnels are already open' error, try restarting the runtime.")


🌍 Your Streamlit app is running at: https://sarita-pseudocoelomate-penultimately.ngrok-free.dev
Click the link above to open the application in a new tab.


In [4]:
!ngrok config add-authtoken 35vcT2FDxR8Mrt95K35plqrv4Sj_3UtsZAb3jUDfCvRHB2MLd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
